In [23]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
import pandas as pd
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding,LSTM
from tensorflow.python.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahmet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')


In [4]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [5]:
train = train.drop('dataset', axis=1)
test = test.drop('dataset', axis=1)

In [9]:
import re
import string
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(
      lowercase, f"[{re.escape(string.punctuation)}]","")

In [10]:
custom_standardization(train.text)
custom_standardization(test.text)

<tf.Tensor: shape=(48965,), dtype=string, numpy=
array([b'kral akbaba dikkat \xc3\xa7ekici renklere sahiptir ',
       b' \xc4\xb1srarla korkutmay\xc4\xb1 ba\xc5\x9far\xc4\xb1yor sanki korku \xc3\xa7ok uzun ve bitmiyor zira film bitiyor ama m\xc3\xbczik hal\xc3\xa2 uyarmaya devam ediyor 9 ',
       b'ne\xc5\x9fe ve \xc3\x9cz\xc3\xbcnt\xc3\xbc k\xc3\xb6pr\xc3\xbcn\xc3\xbcn k\xc4\xb1r\xc4\xb1lmaya ba\xc5\x9flamas\xc4\xb1yla geri d\xc3\xb6nerler ',
       ..., b'kurgunu skiyim o\xc3\xa7 evlad\xc4\xb1',
       b'\xc5\x9eark\xc4\xb1 daha sonras\xc4\xb1nda damian marley taraf\xc4\xb1ndan seslendirildi ',
       b'berrak bir \xc3\xbcr\xc3\xbcn ancak kendi orijinal spigen lyquid crystal k\xc4\xb1l\xc4\xb1fla birlikte kullanmama ra\xc4\x9fmen yan taraftan jelatin kalkt\xc4\xb1 ayni anda ikisini birden kullanamazs\xc4\xb1n\xc4\xb1z'],
      dtype=object)>

In [12]:
def token(values):
    words = nltk.tokenize.word_tokenize(values)
    filtered_words = [word for word in words if word not in stop_word_list]
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc

In [13]:
train['text'] = train['text'].apply(lambda x: token(x))
test['text'] = test['text'].apply(lambda x: token(x))

In [14]:
x_train = train.text
x_test = test.text
y_train = train.label
y_test = test.label

In [18]:
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(train.text)
tokenizer.word_index

{'bir': 1,
 'ürün': 2,
 'iyi': 3,
 'güzel': 4,
 'tavsiye': 5,
 'ederim': 6,
 'film': 7,
 'olarak': 8,
 'kadar': 9,
 'yok': 10,
 'gayet': 11,
 'aldım': 12,
 'sonra': 13,
 'var': 14,
 'ben': 15,
 'teşekkürler': 16,
 'ürünü': 17,
 'hızlı': 18,
 'bi': 19,
 '2': 20,
 'bu': 21,
 'göre': 22,
 'kaliteli': 23,
 'gerçekten': 24,
 'değil': 25,
 'kesinlikle': 26,
 'uygun': 27,
 'cok': 28,
 'bence': 29,
 'tek': 30,
 'gün': 31,
 'filmi': 32,
 '1': 33,
 'biraz': 34,
 'bile': 35,
 'elime': 36,
 'oldu': 37,
 'ilk': 38,
 'geldi': 39,
 '10': 40,
 'ayrıca': 41,
 'hepsiburada': 42,
 'fiyata': 43,
 'fiyat': 44,
 '3': 45,
 'memnun': 46,
 'ancak': 47,
 'önce': 48,
 'büyük': 49,
 'mükemmel': 50,
 'kargo': 51,
 'aynı': 52,
 'olan': 53,
 'sadece': 54,
 'kullanışlı': 55,
 'harika': 56,
 'tam': 57,
 'fazla': 58,
 'iki': 59,
 'zaten': 60,
 'kötü': 61,
 'fakat': 62,
 'olması': 63,
 'telefon': 64,
 'ulaştı': 65,
 'uzun': 66,
 '5': 67,
 '’': 68,
 'kullanıyorum': 69,
 'son': 70,
 'diğer': 71,
 'oldukça': 72,
 'olduğu':

In [27]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
y_train_tokens = tokenizer.texts_to_sequences(y_train)
y_test_tokens = tokenizer.texts_to_sequences(y_test)

In [20]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [21]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)

In [22]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9680992721242372

In [28]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)
y_train_pad = pad_sequences(y_train_tokens, maxlen=max_tokens)
y_test_pad = pad_sequences(y_test_tokens, maxlen=max_tokens)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.optimizers import Adam

# Ardışık bir model oluşturma
model = Sequential()

# Her kelimeye karşılık gelen 50 uzunluğunda bir vektör oluşturulur. (Embedding matrisi)
embedding_size = 50

# Matris kelime sayısı ve embedding büyüklüğünde olacak, yani 10 bine 50 uzunluğunda.
model.add(Embedding(input_dim=10000,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

# LSTM layerlerinin eklenmesi
## 16 nöronlu LSTM (16 outputlu , return_sequences=True demek output'un tamamını ver demek)
model.add(LSTM(units=16, return_sequences=True))
## 8 nöronlu LSTM (8 outputlu , return_sequences=True demek output'un tamamını ver demek)
model.add(LSTM(units=8, return_sequences=True))
## 4 nöronlu LSTM (4 outputlu , return_sequences=False yani default değer, tek bir output verecek)
model.add(LSTM(units=4))
## output layer'ı , görsel olarak gösterilirken dense layer kullanılır.  Tek bir nörondan oluştuğu için 1 yazılır.
model.add(Dense(3, activation='sigmoid'))

# Optimizasyon algoritması, 1e-3 = 0.001 demek.
optimizer = Adam(lr=1e-3)

# Modeli derlemek, loss fonksiyonu binary_crossentropy -> sadece 2 sınıf ama daha fazla sınıflar için categorical_crossentropy kullanılır.
# Metrics -> modelin başarısını görmek için.
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Etiketleri sayısal türe dönüştürme
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

# Etiketleri kategorik kodlama kullanarak uygun formata dönüştürme
y_test_categorical = to_categorical(y_test_encoded)
y_train_categorical = to_categorical(y_train_encoded)

C:\Users\ahmet\anaconda3\envs\eski\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [30]:
# Modeli derlerken kayıp fonksiyonunu 'categorical_crossentropy' olarak belirleme
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


In [ ]:
history = model.fit(x_train_pad , y_train_categorical, validation_data=(x_test_pad , y_test_categorical), epochs=25, batch_size=256)

Epoch 1/25
1722/1722 [==============================] - 211s 122ms/step - loss: 0.2242 - accuracy: 0.9233 - val_loss: 0.1791 - val_accuracy: 0.9348
Epoch 2/25
1722/1722 [==============================] - 208s 121ms/step - loss: 0.1624 - accuracy: 0.9418 - val_loss: 0.1724 - val_accuracy: 0.9362
Epoch 3/25
1722/1722 [==============================] - 211s 123ms/step - loss: 0.1436 - accuracy: 0.9485 - val_loss: 0.1698 - val_accuracy: 0.9376
Epoch 4/25
1722/1722 [==============================] - 212s 123ms/step - loss: 0.1283 - accuracy: 0.9546 - val_loss: 0.1741 - val_accuracy: 0.9371
Epoch 5/25
1722/1722 [==============================] - 213s 124ms/step - loss: 0.1145 - accuracy: 0.9601 - val_loss: 0.1868 - val_accuracy: 0.9336
Epoch 6/25
1722/1722 [==============================] - 215s 125ms/step - loss: 0.1023 - accuracy: 0.9648 - val_loss: 0.1968 - val_accuracy: 0.9328
Epoch 7/25
1722/1722 [==============================] - 214s 124ms/step - loss: 0.0909 - accuracy: 0.9692 - val_